In [1]:
# Import findspark to locate Spark in your Python Env
import findspark
findspark.init()
findspark.find()

# Import SparkSession

from pyspark.sql import SparkSession

# Create SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("PySpark Databricks") \
        .getOrCreate()

print("Spark Session Details:",spark)

Spark Session Details: <pyspark.sql.session.SparkSession object at 0x000002C16169EA10>


In [2]:
# Range toDF
my_range = spark.range(5).toDF("number")
my_range.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
+------+



In [3]:
# Using Where for Evens 
evens = my_range.where("number % 2 = 0")
evens.show()

+------+
|number|
+------+
|     0|
|     2|
|     4|
+------+



In [4]:
# Read CSV with InferSchema and Headers
# Read is a Transformation. Thus, it has Lazy Evaluation

flightData2015 = spark.read \
    .option("inferschema", "true") \
    .option("header","true") \
    .csv("Data_Files\Flights.csv")

flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [5]:
# Explain - Physical Plan

flightData2015.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#33 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#33 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=53]
      +- FileScan csv [DEST_COUNTRY_NAME#31,ORIGIN_COUNTRY_NAME#32,count#33] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/c:/Users/PRATIK/Documents/Practice/PySpark_Practice/Databricks_B..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [6]:
spark.conf.set("spark.sql.shuffle.partitions","5")      # Set to 5, Becoz Default Shuffle Partitions is 200

flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='Ireland', ORIGIN_COUNTRY_NAME='Afganistan', count=2)]

In [7]:
flightData2015.rdd.getNumPartitions()

1

In [8]:
flightData2015.createOrReplaceTempView("flights_data_2015")

# SQL Way
flights_sql_res = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flights_data_2015
GROUP BY DEST_COUNTRY_NAME""")

flights_sql_res.show()

# Dataframe Way
flights_df_res = flightData2015 \
    .groupBy("DEST_COUNTRY_NAME") \
    .count()

flights_df_res.show()

# It does not matter which way we use, final plan which Spark complies remains the same
flights_sql_res.explain()
flights_df_res.explain()

+-----------------+--------+
|DEST_COUNTRY_NAME|count(1)|
+-----------------+--------+
|   United Kingdom|       2|
|          Ireland|       1|
|          Germany|       1|
|           Russia|       1|
|    United States|       3|
|            India|       1|
+-----------------+--------+

+-----------------+-----+
|DEST_COUNTRY_NAME|count|
+-----------------+-----+
|   United Kingdom|    2|
|          Ireland|    1|
|          Germany|    1|
|           Russia|    1|
|    United States|    3|
|            India|    1|
+-----------------+-----+

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#31], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#31, 5), ENSURE_REQUIREMENTS, [plan_id=160]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#31], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#31] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/c:/Users/PRA

In [9]:
# Max
# SQL
spark.sql(""" SELECT MAX(count) FROM flights_data_2015 """).show()

# DF
from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1)

+----------+
|max(count)|
+----------+
|       344|
+----------+



[Row(max(count)=344)]

In [10]:
max_sql = spark.sql("""
SELECT DEST_COUNTRY_NAME, SUM(count) AS Destination_Total
FROM flights_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY SUM(count) DESC
LIMIT 5
""")

max_sql.collect()

[Row(DEST_COUNTRY_NAME='United States', Destination_Total=360),
 Row(DEST_COUNTRY_NAME='United Kingdom', Destination_Total=25),
 Row(DEST_COUNTRY_NAME='Germany', Destination_Total=10),
 Row(DEST_COUNTRY_NAME='Russia', Destination_Total=5),
 Row(DEST_COUNTRY_NAME='India', Destination_Total=5)]

In [11]:
from pyspark.sql.functions import desc

max_df = flightData2015 \
    .groupBy("DEST_COUNTRY_NAME") \
    .sum("count") \
    .withColumnRenamed("sum(count)", "destination_total") \
    .sort(desc("destination_total")) \
    .limit(5)

max_df.explain()

max_df.collect()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#121L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#31,destination_total#121L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#31], functions=[sum(count#33)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#31, 5), ENSURE_REQUIREMENTS, [plan_id=298]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#31], functions=[partial_sum(count#33)])
            +- FileScan csv [DEST_COUNTRY_NAME#31,count#33] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/c:/Users/PRATIK/Documents/Practice/PySpark_Practice/Databricks_B..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




[Row(DEST_COUNTRY_NAME='United States', destination_total=360),
 Row(DEST_COUNTRY_NAME='United Kingdom', destination_total=25),
 Row(DEST_COUNTRY_NAME='Germany', destination_total=10),
 Row(DEST_COUNTRY_NAME='Russia', destination_total=5),
 Row(DEST_COUNTRY_NAME='India', destination_total=5)]

In [12]:
#  Spark Streaming Example

staticDataFrame = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("Data_Files/Retail.csv")

staticDataFrame.createOrReplaceTempView("retail_data")

staticSchema = staticDataFrame.schema
staticDataFrame.printSchema()

root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [13]:
spark.sql("""
SELECT DISTINCT DATE(InvoiceDate), COUNT(*)
FROM retail_data
GROUP BY 1
ORDER BY 2 DESC
""").show()

+-----------+--------+
|InvoiceDate|count(1)|
+-----------+--------+
| 2010-12-01|       6|
| 2010-12-03|       3|
| 2010-12-05|       1|
| 2010-12-02|       1|
+-----------+--------+



In [14]:
# Windowing per day based on a Business Column

from pyspark.sql.functions import window, column, col, desc

staticDataFrame \
.selectExpr(
    "CustomerId",
    "(UnitPrice*Quantity) as total_cost",
    "InvoiceDate")\
.groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")\
.show(truncate=False)

+----------+------------------------------------------+------------------+
|CustomerId|window                                    |sum(total_cost)   |
+----------+------------------------------------------+------------------+
|17850.0   |{2010-12-01 05:30:00, 2010-12-02 05:30:00}|112.92000000000002|
|17850.0   |{2010-12-03 05:30:00, 2010-12-04 05:30:00}|15.3              |
|17850.0   |{2010-12-05 05:30:00, 2010-12-06 05:30:00}|11.100000000000001|
|13047.0   |{2010-12-03 05:30:00, 2010-12-04 05:30:00}|108.16            |
|17850.0   |{2010-12-02 05:30:00, 2010-12-03 05:30:00}|22.0              |
+----------+------------------------------------------+------------------+



In [15]:
# Converting above Batch logic to Stream
# Won't Run, but keeping the Syntax for References
"""
# Reading Stream
streamingDataFrame = spark.readStream\
 .schema(staticSchema)\     # Providing Schema of Streaming Data
 .option("maxFilesPerTrigger", 1)\
 .format("csv")\
 .option("header", "true")\
 .load(".../*.csv")

streamingDataFrame.isStreaming      # returns true

purchaseByCustomerPerHour = streamingDataFrame\
 .selectExpr(
 "CustomerId",
 "(UnitPrice * Quantity) as total_cost" ,
 "InvoiceDate" )\
 .groupBy(
 col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
 .sum("total_cost")

# But to Run, we have to call an Action
# Streaming has different actions
purchaseByCustomerPerHour.writeStream\
 .format("memory")\     #  memory = store in-memory table | For writing to Console >> .format("console")
 .queryName("customer_purchases")\      # customer_purchases = name of the in-memory table
 .outputMode("complete")\       #  complete = all the counts should be in the table
 .start()
"""

'\n# Reading Stream\nstreamingDataFrame = spark.readStream .schema(staticSchema)\\     # Providing Schema of Streaming Data\n .option("maxFilesPerTrigger", 1) .format("csv") .option("header", "true") .load(".../*.csv")\n\nstreamingDataFrame.isStreaming      # returns true\n\npurchaseByCustomerPerHour = streamingDataFrame .selectExpr(\n "CustomerId",\n "(UnitPrice * Quantity) as total_cost" ,\n "InvoiceDate" ) .groupBy(\n col("CustomerId"), window(col("InvoiceDate"), "1 day")) .sum("total_cost")\n\n# But to Run, we have to call an Action\n# Streaming has different actions\npurchaseByCustomerPerHour.writeStream .format("memory")\\     #  memory = store in-memory table | For writing to Console >> .format("console")\n .queryName("customer_purchases")\\      # customer_purchases = name of the in-memory table\n .outputMode("complete")\\       #  complete = all the counts should be in the table\n .start()\n'

In [16]:
# Lower Level APIs - RDD
# Parallelizing RDD into a DF

from pyspark.sql import Row

spark.sparkContext.parallelize([Row(1), Row(2), Row(3)]).toDF()\
    .withColumnRenamed("_1", "Numbers")\
    .show()

+-------+
|Numbers|
+-------+
|      1|
|      2|
|      3|
+-------+



In [17]:
# lit Function - Converting to Spark Native Type

from pyspark.sql.functions import lit

df = spark.read.format("csv")\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .load("Data_Files/Retail.csv")

df.select(lit(5), lit("five"), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

In [18]:
# using Where

# Way 1 using col()

df.where(col("InvoiceNo") != 536365)\
    .select("InvoiceNo", "Description")\
    .show(5, False)

# Way 2 using Expression String
df.where("InvoiceNo != 536365")\
    .select("InvoiceNo", "Description")\
    .show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |null                         |
+---------+-----------------------------+

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |null                         |
+---------+-----------------------------+



In [19]:
# More Difficult Where Example
from pyspark.sql.functions import instr

priceFilter = col("Unitprice") > 1.85
descriptionFilter = instr(df.Description, "WHITE") >= 1

# col("StockCode") and df.StockCode - Ways to access a column of Dataframe
# where and filter - Are Alias in case of DF. Both works similarly for Dataframes

df.where(col("StockCode").isin("71053"))\
    .where(priceFilter | descriptionFilter)\
    .show()

df.filter(df.StockCode.isin("71053"))\
    .filter(priceFilter | descriptionFilter)\
    .show()

+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|        Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|   536365|    71053|WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+

+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|        Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|   536365|    71053|WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+---------------

In [20]:
spark.sql("""
SELECT *
FROM retail_data
WHERE 
StockCode in ("71053") AND
(UnitPrice > 1.85 OR instr(Description, "WHITE") >= 1 )
""").show()

+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|        Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|   536365|    71053|WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+



In [21]:
# Where vs Filter
"""
Where - 
- is for Dataframes
- SQL like conditions
- Alias for Filter
Filter 
- for Dataframes and RDDs
- Supports lamda function for advanced filters
"""

# Use & for and, | for or, ~ for not In DF boolean expressions
DOTCodeFilter = col("StockCode") == "71053"
priceFilter = col("UnitPrice") > 1.85
descripFilter = instr(col("Description"), "WHITE") >= 1
df.withColumn("isExpensive",
DOTCodeFilter & (priceFilter | descripFilter))\
.where("isExpensive")\
.select("unitPrice", "isExpensive")\
.show(5)

spark.sql("""
SELECT
UnitPrice,
(StockCode = "71053" AND
(UnitPrice > 1.85 OR
instr(Description, "WHITE") >= 1)) as isExpensive
FROM retail_data
""").show()

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|     3.39|       true|
+---------+-----------+

+---------+-----------+
|UnitPrice|isExpensive|
+---------+-----------+
|     2.55|      false|
|     3.39|       true|
|     2.75|      false|
|     3.39|      false|
|     3.39|      false|
|     7.65|      false|
|     4.25|      false|
|     1.85|      false|
|     1.85|      false|
|     1.69|      false|
|     1.69|      false|
+---------+-----------+



In [22]:
from pyspark.sql.functions import expr, col

df.withColumn("isExpensive", expr("NOT UnitPrice <= 1.85"))\
    .where(col("isExpensive"))\
    .select("Description", "UnitPrice", "isExpensive").show()

+--------------------+---------+-----------+
|         Description|UnitPrice|isExpensive|
+--------------------+---------+-----------+
|RED WHITE HANGING...|     2.55|       true|
| WHITE METAL LANTERN|     3.39|       true|
|CREAM CUPID HEART...|     2.75|       true|
|KNITTED UNION FLA...|     3.39|       true|
|RED WOOLLY HOTTIE...|     3.39|       true|
|SET 7 BABUSHKA NE...|     7.65|       true|
|GLASS STAR FROSTE...|     4.25|       true|
+--------------------+---------+-----------+



In [23]:
# To perform a null safe equivalence test
df.where(col("Description").eqNullSafe("hello")).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [24]:
# Mathematical Examples
# pow and expr
from pyspark.sql.functions import expr, pow

fabricatedQty = pow( (col("Quantity") * col("UnitPrice")), 2) + 5
df.select(
    expr("CustomerId"),
    fabricatedQty.alias("RealQty")
).show(3)

# Other Way
df.selectExpr(
    "CustomerId",
    "(POWER((Quantity*UnitPrice),2)+5) as RealQty"
).show(3)

+----------+------------------+
|CustomerId|           RealQty|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
+----------+------------------+
only showing top 3 rows

+----------+------------------+
|CustomerId|           RealQty|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
|   17850.0|             489.0|
+----------+------------------+
only showing top 3 rows



In [25]:
# Round, BRound
from pyspark.sql.functions import round, bround, lit

df.select(
    round(lit("2.5")),      # Round to Ceil
    bround(lit("2.5")),     # Round to Bottom when Exact Centre Number is Rounder eg. 2.5 
    round(lit("2.4")),
    bround(lit("2.6"))
).show(2)

+-------------+--------------+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|round(2.4, 0)|bround(2.6, 0)|
+-------------+--------------+-------------+--------------+
|          3.0|           2.0|          2.0|           3.0|
|          3.0|           2.0|          2.0|           3.0|
+-------------+--------------+-------------+--------------+
only showing top 2 rows



In [26]:
# Describes a DF
df.describe().show()

+-------+------------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|summary|         InvoiceNo|         StockCode|         Description|          Quantity|         UnitPrice|        CustomerID|       Country|
+-------+------------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|  count|                11|                11|                  10|                11|                11|                11|            11|
|   mean| 536365.5454545454| 47222.57142857143|                null|10.545454545454545|3.1318181818181823|16976.727272727272|          null|
| stddev|0.8201995322555489|31256.801504404033|                null|10.699192832766744|1.7260464546576852|1942.9091636465712|          null|
|    min|            536365|             21730|ASSORTED COLOUR B...|                 2|              1.69|           13047.0|United Kingdom|
|    max|    

In [27]:
# Monotonically Increasing ID
from pyspark.sql.functions import monotonically_increasing_id

df.select(monotonically_increasing_id()).show(5)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                            3|
|                            4|
+-----------------------------+
only showing top 5 rows



In [28]:
# String Functions - initcap, lower, upper
from pyspark.sql.functions import initcap, lower, upper

df.select(initcap(lit("raw mango")), 
    lower(lit("APPLE")), 
    upper(lit("orange")))\
    .show(1)

+------------------+------------+-------------+
|initcap(raw mango)|lower(APPLE)|upper(orange)|
+------------------+------------+-------------+
|         Raw Mango|       apple|       ORANGE|
+------------------+------------+-------------+
only showing top 1 row



In [29]:
# String Functions - lpad, rpad, ltrim, rtrim, trim
from pyspark.sql.functions import lpad, rpad, ltrim, rtrim, trim

df.select(
    ltrim(lit("  APPLE  ")).alias("ltrim"),
    rtrim(lit("  APPLE  ")).alias("rtrim"),
    trim(lit("  APPLE  ")).alias("trim"),
    lpad(lit("APPLE"), 8, "-").alias("lp8"),
    lpad(lit("HELLO"), 3, "-").alias("lp3"),
    rpad(lit("APPLE"), 8, "-").alias("rp8"),
    rpad(lit("HELLO"), 3, "-").alias("rp3"),
).show(1)

# Note -  lpad or rpad takes a number less than the length of the string, it 
# will always remove values from the right side of the string.


+-------+-------+-----+--------+---+--------+---+
|  ltrim|  rtrim| trim|     lp8|lp3|     rp8|rp3|
+-------+-------+-----+--------+---+--------+---+
|APPLE  |  APPLE|APPLE|---APPLE|HEL|APPLE---|HEL|
+-------+-------+-----+--------+---+--------+---+
only showing top 1 row



In [30]:
# Regular Expressions
from pyspark.sql.functions import regexp_replace, regexp_extract, translate

df.select(
    regexp_replace(col("Description"), "WHITE|RED", "COLOR").alias("ColorClean"),
    col("Description"))\
    .where("Description LIKE '%RED%' OR Description LIKE '%WHITE%'")\
    .show(5)

+--------------------+--------------------+
|          ColorClean|         Description|
+--------------------+--------------------+
|COLOR COLOR HANGI...|RED WHITE HANGING...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
|COLOR WOOLLY HOTT...|RED WOOLLY HOTTIE...|
|HAND WARMER COLOR...|HAND WARMER RED P...|
+--------------------+--------------------+



In [31]:
df.select(
    translate(col("Description"), "LEET", "1337").alias("CharReplaced"),
    col("Description")
).show(5)

+--------------------+--------------------+
|        CharReplaced|         Description|
+--------------------+--------------------+
|R3D WHI73 HANGING...|RED WHITE HANGING...|
| WHI73 M37A1 1AN73RN| WHITE METAL LANTERN|
|CR3AM CUPID H3AR7...|CREAM CUPID HEART...|
|KNI773D UNION F1A...|KNITTED UNION FLA...|
|R3D WOO11Y HO77I3...|RED WOOLLY HOTTIE...|
+--------------------+--------------------+
only showing top 5 rows



In [32]:
# Pulling out 1st occurance of color
regexp_str = "(RED|WHITE|GREEN)"
df.select(
    regexp_extract(col("Description"), regexp_str, 1).alias("FirstColor"),
    col("Description")
).show(5)

+----------+--------------------+
|FirstColor|         Description|
+----------+--------------------+
|       RED|RED WHITE HANGING...|
|     WHITE| WHITE METAL LANTERN|
|          |CREAM CUPID HEART...|
|          |KNITTED UNION FLA...|
|       RED|RED WOOLLY HOTTIE...|
+----------+--------------------+
only showing top 5 rows



In [33]:
# instr : To identify the occurance of a substring in a string

red_filter = instr(col("Description"), "RED") >= 1
white_filter = instr(col("Description"), "WHITE") >= 1
df.withColumn("hasColor", red_filter | white_filter)\
    .filter("hasColor")\
    .select(col("Description"))\
    .show(5)

+--------------------+
|         Description|
+--------------------+
|RED WHITE HANGING...|
| WHITE METAL LANTERN|
|RED WOOLLY HOTTIE...|
|HAND WARMER RED P...|
+--------------------+



In [34]:
# Current Date and Timestamp Functions
from pyspark.sql.functions import current_date, current_timestamp

dateDF = spark.range(1)\
    .withColumn("date", current_date())\
    .withColumn("datetime", current_timestamp())

print(dateDF.count())

dateDF.createOrReplaceTempView("dateTable")

1


In [35]:
dateDF.show(truncate=False)

spark.sql("""
SELECT
    current_date() AS date,
    current_timestamp() AS now
""").show(truncate=False)

+---+----------+--------------------------+
|id |date      |datetime                  |
+---+----------+--------------------------+
|0  |2024-04-20|2024-04-20 17:20:38.975437|
+---+----------+--------------------------+

+----------+--------------------------+
|date      |now                       |
+----------+--------------------------+
|2024-04-20|2024-04-20 17:20:39.019462|
+----------+--------------------------+



In [36]:
from pyspark.sql.functions import date_sub, date_add, datediff, months_between, to_date

date_diffDF = dateDF.select(
    date_sub("date", 5).alias("5less"),
    date_add("date", 5).alias("5more")
)

date_diffDF.show()

date_diffDF.select(
    datediff("5less", "5more").alias("Less-More"),
    datediff("5more", "5less").alias("More-Less")
).show()

# Used spark.range(1) for a Temp Dataframe

spark.range(1)\
    .select(
    to_date(lit("2024-01-20")).alias("start"),
    to_date(lit("2024-06-20")).alias("end"))\
    .select(months_between(col("end"), col("start")))\
    .show(1)

+----------+----------+
|     5less|     5more|
+----------+----------+
|2024-04-15|2024-04-25|
+----------+----------+

+---------+---------+
|Less-More|More-Less|
+---------+---------+
|      -10|       10|
+---------+---------+

+--------------------------------+
|months_between(end, start, true)|
+--------------------------------+
|                             5.0|
+--------------------------------+



In [37]:
to_date_DF = spark.range(1)\
    .withColumn("str_date", lit("2024-05-04"))\
    .withColumn("casted_date", to_date(col("str_date")))

to_date_DF.show()

to_date_DF.printSchema()        # Notice Types in Schema

+---+----------+-----------+
| id|  str_date|casted_date|
+---+----------+-----------+
|  0|2024-05-04| 2024-05-04|
+---+----------+-----------+

root
 |-- id: long (nullable = false)
 |-- str_date: string (nullable = false)
 |-- casted_date: date (nullable = true)



In [38]:
# Spark will return NULL if it is not able to parse the dates
# Explicit Format and Cast is Recommended over Implicit for date formats
# Default Format = yyyy-MM-dd

spark.range(1)\
    .select(to_date(lit("2020-20-10")),     # NULL bcoz format difference
    to_date(lit("2020-12-11"))              # As in data format is yyyy-dd-MM, this was 12th Nov, but shown as 11th Dec
    ).show()

# Standardizing the fromat
date_format = "yyyy-dd-MM"

spark.range(1)\
    .select(to_date(lit("2020-20-10"), date_format),
    to_date(lit("2020-12-11"), date_format)
    ).show()


+-------------------+-------------------+
|to_date(2020-20-10)|to_date(2020-12-11)|
+-------------------+-------------------+
|               null|         2020-12-11|
+-------------------+-------------------+

+-------------------------------+-------------------------------+
|to_date(2020-20-10, yyyy-dd-MM)|to_date(2020-12-11, yyyy-dd-MM)|
+-------------------------------+-------------------------------+
|                     2020-10-20|                     2020-11-12|
+-------------------------------+-------------------------------+



In [39]:
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_timestamp

# Human to Epoch Time
epoch_time_df = spark.range(1)\
    .select(
    to_timestamp(lit("2020-04-21 00:00:00")).alias("human_time"),
    unix_timestamp(lit("2020-04-21 00:00:00")).alias("epoch_time"))

epoch_time_df.show()

# Epoch Time to Human Time
epoch_time_df\
    .select(from_unixtime(col("epoch_time")).alias("fetched_human_time"),
    col("epoch_time")
    ).show()

# Casting to_timestamp
spark.range(1)\
    .select(
        to_timestamp(lit("2022-12-14"), "yyyy-MM-dd")
    ).show()

+-------------------+----------+
|         human_time|epoch_time|
+-------------------+----------+
|2020-04-21 00:00:00|1587407400|
+-------------------+----------+

+-------------------+----------+
| fetched_human_time|epoch_time|
+-------------------+----------+
|2020-04-21 00:00:00|1587407400|
+-------------------+----------+

+------------------------------------+
|to_timestamp(2022-12-14, yyyy-MM-dd)|
+------------------------------------+
|                 2022-12-14 00:00:00|
+------------------------------------+



In [40]:
# Once dates are in proper format, they can be compared easily
spark.range(1)\
    .selectExpr("""to_date("2022-12-11") > to_date("2022-12-14")""")\
    .show()

+-------------------------------------------+
|(to_date(2022-12-11) > to_date(2022-12-14))|
+-------------------------------------------+
|                                      false|
+-------------------------------------------+



In [41]:
# Working with Nulls

from pyspark.sql.functions import coalesce  #coalesce returns 1st Non NULL Value

df.filter(col("StockCode")==84879)\
.select(
    coalesce(col("Description"), col("Customerid")),
    col("Description"),
    col("CustomerId")
).show()

+---------------------------------+--------------------+----------+
|coalesce(Description, Customerid)|         Description|CustomerId|
+---------------------------------+--------------------+----------+
|             ASSORTED COLOUR B...|ASSORTED COLOUR B...|   13047.0|
|                          13047.0|                null|   13047.0|
+---------------------------------+--------------------+----------+



In [46]:
# ifnull = Returns 2nd value if 1st is Null. Defaults to 1st
# nullif = Returns Null if 1st and 2nd value match. Defaults to 1st
# nvl = Return 2nd value if 1st is Null. Defaults to 1st
# nvl2 = Returns 2nd value if 1st is Not Null. Defaults to Else value

spark.sql("""
SELECT
    ifnull(null, "return_value"),
    ifnull("not_null", "return_value"),
    nullif("value", "value"),
    nullif("value1", "value2"),
    nvl(null, "return_value"),
    nvl("not_null", "return_value"),
    nvl2("not_null", "return_value", "else_val"),
    nvl2(null, "return_value", "else_val")
"""
).show()

# Generally Used
# IFNULL, COALESCE

+--------------------------+------------------------------+--------------------+----------------------+-----------------------+---------------------------+--------------------------------------+----------------------------------+
|ifnull(NULL, return_value)|ifnull(not_null, return_value)|nullif(value, value)|nullif(value1, value2)|nvl(NULL, return_value)|nvl(not_null, return_value)|nvl2(not_null, return_value, else_val)|nvl2(NULL, return_value, else_val)|
+--------------------------+------------------------------+--------------------+----------------------+-----------------------+---------------------------+--------------------------------------+----------------------------------+
|              return_value|                      not_null|                null|                value1|           return_value|                   not_null|                          return_value|                          else_val|
+--------------------------+------------------------------+--------------------+

In [67]:
null_df = spark.read.format("csv")\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .load("Data_Files/NullsData.csv")

null_df.show()

+----+-------+-----+
|  id|   name|place|
+----+-------+-----+
|   1|    ram|   p1|
|   3|   shiv|   p3|
|null| pratik| null|
|   6|   null| null|
|   4|hanuman|   p4|
+----+-------+-----+



In [63]:
# all = Drop only if all columns in a row have nulls
# any/blank = Drop if any column in a row have null


drop_df = null_df.na.drop()
drop_df.show()

drop_df1 = null_df.na.drop("any")
drop_df1.show()

drop_df2 = null_df.na.drop("all")
drop_df2.show()

+---+-------+-----+
| id|   name|place|
+---+-------+-----+
|  1|    ram|   p1|
|  3|   shiv|   p3|
|  4|hanuman|   p4|
+---+-------+-----+

+---+-------+-----+
| id|   name|place|
+---+-------+-----+
|  1|    ram|   p1|
|  3|   shiv|   p3|
|  4|hanuman|   p4|
+---+-------+-----+

+----+-------+-----+
|  id|   name|place|
+----+-------+-----+
|   1|    ram|   p1|
|   3|   shiv|   p3|
|null| pratik| null|
|   6|   null| null|
|   4|hanuman|   p4|
+----+-------+-----+



In [66]:
# All with Subset
drop_df3 = null_df.na.drop("all", subset=["name", "place"])     # id=6 is that case
drop_df3.show()

+----+-------+-----+
|  id|   name|place|
+----+-------+-----+
|   1|    ram|   p1|
|   3|   shiv|   p3|
|null| pratik| null|
|   4|hanuman|   p4|
+----+-------+-----+



In [73]:
null_df.show()

null_df.na.fill("Fillers").show()   #Fills all String Columns with Specified value

+----+-------+-----+
|  id|   name|place|
+----+-------+-----+
|   1|    ram|   p1|
|   3|   shiv|   p3|
|null| pratik| null|
|   6|   null| null|
|   4|hanuman|   p4|
+----+-------+-----+

+----+-------+-------+
|  id|   name|  place|
+----+-------+-------+
|   1|    ram|     p1|
|   3|   shiv|     p3|
|null| pratik|Fillers|
|   6|Fillers|Fillers|
|   4|hanuman|     p4|
+----+-------+-------+



In [81]:
null_df.na.fill("Filler_Val", subset=["name", "place"]).show()      # Filing for subset of columns

+----+----------+----------+
|  id|      name|     place|
+----+----------+----------+
|   1|       ram|        p1|
|   3|      shiv|        p3|
|null|    pratik|Filler_Val|
|   6|Filler_Val|Filler_Val|
|   4|   hanuman|        p4|
+----+----------+----------+



In [83]:
# Filling values from a Map
fillers = {
    "id":10,
    "name":"TestName"
}

null_df.na.fill(fillers).show()     # Filling specified default values from a Map/Dictionary

+---+--------+-----+
| id|    name|place|
+---+--------+-----+
|  1|     ram|   p1|
|  3|    shiv|   p3|
| 10|  pratik| null|
|  6|TestName| null|
|  4| hanuman|   p4|
+---+--------+-----+



In [84]:
# Replace in a Column (value_to_replace, replacement_value, in_which_column)
null_df.na.replace("pratik", "replaced_pratik", "name").show()

+----+---------------+-----+
|  id|           name|place|
+----+---------------+-----+
|   1|            ram|   p1|
|   3|           shiv|   p3|
|null|replaced_pratik| null|
|   6|           null| null|
|   4|        hanuman|   p4|
+----+---------------+-----+



In [88]:
# Ordering in Dataframe

null_df.orderBy(col("id").asc()).select("*").show()

null_df.orderBy(col("id").desc()).select("*").show()

+----+-------+-----+
|  id|   name|place|
+----+-------+-----+
|null| pratik| null|
|   1|    ram|   p1|
|   3|   shiv|   p3|
|   4|hanuman|   p4|
|   6|   null| null|
+----+-------+-----+

+----+-------+-----+
|  id|   name|place|
+----+-------+-----+
|   6|   null| null|
|   4|hanuman|   p4|
|   3|   shiv|   p3|
|   1|    ram|   p1|
|null| pratik| null|
+----+-------+-----+

